In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.utils.data as Data
%matplotlib inline
plt.style.use('ggplot')

def training_data_describe(training_set_path):
    training_set=pd.read_csv(training_set_path)
    training_data_describe=pd.DataFrame()
    training_data_describe["std"]=round(training_set.std(),3)
    training_data_describe["mean"]=round(training_set.mean(),3)
    training_data_describe["max"]=round(training_set.max(),3)
    training_data_describe["min"]=round(training_set.min(),3)
    training_data_describe=training_data_describe.T
    training_data_describe.index.name="describe"
    training_data_describe.to_csv("data_describ_2y.csv")
    return training_data_describe

def data_norm(df):
    df_norm=df
    for i in df.columns:
        df_norm[i]=(df_norm[i]-training_data_describe.loc['mean',i])/training_data_describe.loc['std',i]
    return df_norm

def buildWindows(data, windowsize=10,feature=5):
    X_Window, Y_Window = [], []
    for i in range(data.shape[0]-windowsize+1):
        data_select=data.iloc[i:i+windowsize,0:feature:].drop_duplicates()
        if data_select.shape[0]==data.iloc[i:i+windowsize,0:feature:].shape[0]:
            Y_Window.append(np.array(data.iloc[i+past:i+windowsize,feature:]))
            X_Window.append(np.array(data.iloc[i:i+windowsize,0:feature])) 
    return np.array(X_Window), np.array(Y_Window)

def window_p(training_set_norm,mode):
    X_train_W, Y_train_W = buildWindows(training_set_norm, WindowsSize,featureNum)
    if mode=="Train":
        X_train, Y_train, X_val, Y_val = splitData(X_train_W, Y_train_W, 0.8)
        return X_train, Y_train, X_val, Y_val
    if mode=="Test":
        return X_train_W, Y_train_W

def splitData(X,Y,val_size,mode):
    if mode=="random":
        from sklearn.model_selection import train_test_split
        X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=val_size, random_state=42)
    if mode=="normal":
        X_train = X[:int(X.shape[0]*(1-val_size))]
        Y_train = Y[:int(Y.shape[0]*(1-val_size))]
        X_val = X[int(X.shape[0]*(1-val_size)):]
        Y_val = Y[int(Y.shape[0]*(1-val_size)):]
    return X_train, Y_train, X_val, Y_val

def data_plot(rawdata):
    plt.style.use('ggplot')
    column_name=rawdata.columns
    column_num=len(column_name)
    column_p=[]
    plt.figure(figsize=(50,column_num*10)).patch.set_facecolor('white')
    data_P=rawdata
    for idx, i in enumerate(column_name):
        plt.subplot(column_num,1,idx+1)
        plt.title(i,fontsize=50)
        plt.scatter(rawdata.index,rawdata[i],s=5,c="#56B4E9")
        plt.yticks(fontsize=40)
        plt.xticks(fontsize=40,rotation=45)
    plt.tight_layout()

In [13]:
#prepare torch data
class PrepareData(Data.Dataset):
    def __init__(self, X, y):
        if not torch.is_tensor(X):
            self.X = torch.from_numpy(X)
        if not torch.is_tensor(y):
            self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [1]:
#LSTM model
import torch
import torch.nn as nn
import torch.nn.functional as F
class softsensor(nn.Module):
    def __init__(self,input_feature_dim,hidden_feature_dim1,hidden_feature_dim2,classes_num,gpu):
        super(softsensor, self).__init__()
        
        self.input_feature_dim=input_feature_dim
        self.hidden_feature_dim1=hidden_feature_dim1
        self.hidden_feature_dim2=hidden_feature_dim2
        
        #Initialization        
        self.lstm1=nn.LSTMCell(input_feature_dim,hidden_feature_dim1,bias=False)   
        self.lstm2=nn.LSTMCell(hidden_feature_dim1,hidden_feature_dim2,bias=False) 
        self.linear1=nn.Linear(hidden_feature_dim2,classes_num,bias=False)
        
    def init_hidden(self,batch_size,gpu):
        if gpu==True:
            h01=torch.zeros(batch_size,self.hidden_feature_dim1).cuda()
            c01=torch.zeros(batch_size,self.hidden_feature_dim1).cuda()
            h02=torch.zeros(batch_size,self.hidden_feature_dim2).cuda()
            c02=torch.zeros(batch_size,self.hidden_feature_dim2).cuda()
        else:
            h01=torch.zeros(batch_size,self.hidden_feature_dim1)
            c01=torch.zeros(batch_size,self.hidden_feature_dim1)
            h02=torch.zeros(batch_size,self.hidden_feature_dim2)
            c02=torch.zeros(batch_size,self.hidden_feature_dim2)
        
        return (h01,c01,h02,c02)
  
    def forward(self,input,batchsize,gpu):
        h01,c01,h02,c02=self.init_hidden(batchsize,gpu)
        h1=h01
        c1=c01
        h2=h02
        c2=c02
        output = []
        for i in range(input.size()[1]):
            xt=input[:,i,:]
            h1,c1=self.lstm1(xt.float(),(h01,c01))
            h1=F.tanh(h1)
            #output=F.dropout(output, p=0.1,training=self.training)
            h2,c2=self.lstm2(h1,(h2,c2))
            h2=F.tanh(h2)
            #output=F.dropout(output, p=0.1,training=self.training)
            output.append(h2)
        output=torch.stack(output)
        output=output.permute(1, 0, 2)
        linear_output=self.linear1(output[:,past:])
        return linear_output
    
    def mpc_test(self,input,batchsize,gpu):
            h01,c01,h02,c02=self.init_hidden(batchsize,gpu)
            h1=h01
            c1=c01
            h2=h02
            c2=c02
            outputs = []
            y=[]
            for i in range(input.size()[1]):
                if i<=(past-1):
                    xt=input[:,i,:]      
                    h1,c1=self.lstm1(xt.float(),(h01,c01))
                    #h1=self.BN1(h1)
                    h1=F.tanh(h1)
                    h2,c2=self.lstm2(h1,(h2,c2))
                    #h2=self.BN2(h2)
                    h2=F.tanh(h2)
                    output=self.linear1(h2)
                    outputs.append(output)
                    y=output
                if i>(past-1):
                    xt=input[:,i,:]

                    xt[0,3]=y[0][0]
                    xt[0,4]=y[0][1]
                    h1,c1=self.lstm1(xt.float(),(h01,c01))
                    #h1=self.BN1(h1)
                    h1=F.tanh(h1)
                    h2,c2=self.lstm2(h1,(h2,c2))
                    #h2=self.BN2(h2)
                    h2=F.tanh(h2)
                    output=self.linear1(h2)
                    outputs.append(output)
                    y=output

            outputs=torch.stack(outputs)
            return outputs[past:,:]

In [107]:
# import EarlyStopping
from pytorchtools import EarlyStopping
#Train the Model using Early Stopping
def train_model(model, batch_size, patience, n_epochs,gpu):
    if gpu==True:
        model.to('cuda:0')
        
    # to track the training loss as the model trains
    train_losses = []
    # to track the validation loss as the model trains
    valid_losses = []
    # to track the average training loss per epoch as the model trains
    avg_train_losses = []
    # to track the average validation loss per epoch as the model trains
    avg_valid_losses = [] 
    
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    
    for epoch in range(1, n_epochs + 1):
 
        ###################
        # train the model #
        ###################
        model.train() # prep model for training
        print("training")
        for batch, (data, target) in enumerate(train_loader, 1):
            
            if gpu==True:
                data=data.cuda()
                target=target.cuda()
            else:
                data=data
                target=target
            
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data.float().cuda(), batch_size,gpu=True)
            # calculate the loss
            trainloss = criterion(output.double(), target.double())
            # backward pass: compute gradient of the loss with respect to model parameters
            trainloss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # record training loss
            train_losses.append(trainloss.item())
 
        ######################    
        # validate the model #
        ######################
        model.eval() # prep model for evaluation
        print("validation")
        for data, target in valid_loader:
            
            if gpu==True:
                data=data.cuda()
                target=target.cuda()
            else:
                data=data
                target=target
            
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data.float(), batch_size,gpu=True)
            # calculate the loss
            valloss = criterion(output.double(), target.double())

            # record validation loss
            valid_losses.append(valloss.item())
            
        # print training/validation statistics 
        # calculate average loss over an epoch
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)
        
        epoch_len = len(str(n_epochs))
        
        print_msg = (f'[{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'valid_loss: {valid_loss:.5f}')
        
        print(print_msg)
        
        # clear lists to track next epoch
        train_losses = []
        valid_losses = []
        
        # early_stopping needs the validation loss to check if it has decresed, 
        # and if it has, it will make a checkpoint of the current model
        early_stopping(valid_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
              
        scheduler.step(valloss)
    # load the last checkpoint with the best model
    model.load_state_dict(torch.load('checkpoint.pt'))
 
    return  model, avg_train_losses, avg_valid_losses

In [76]:
def model_test(model,test_loader,i,mode):
    model.cpu()
    model.eval()
    for n,(x, y) in enumerate(test_loader):
        if n==i:
            if mode=="inference":
                y_pred=model(x.float(),1,gpu=False).detach().numpy().squeeze()
            if mode=="mpc_test":
                y_pred=model.mpc_test(x.float(),1,gpu=False).detach().numpy().squeeze()
                
            y_name=["Water","IPAC"]
            y_predicted=pd.DataFrame(np.array(y_pred),columns=[y_name])
            Y_testing=pd.DataFrame(y.detach().numpy().squeeze(),columns=[y_name])
    for i , element in enumerate(y_name):
        from sklearn.metrics import r2_score
        fig = plt.figure(figsize=(8,6))
        plt.subplot(2,len(y_name)/2,i+1)
        plt.title(element)
        plt.scatter( y_predicted[[element]].index,y_predicted[[element]], color='r',s=2)
        plt.scatter( Y_testing[[element]].index,Y_testing[[element]], color='g',s=2)
        plt.legend(['predicted','real'],fontsize=15)
        print("{} R2:".format(element),r2_score(Y_testing[[element]],y_predicted[[element]]))
        plt.tight_layout()
        plt.show()
    return y_predicted

In [ ]:
#prepare training data
training_data_describe=training_data_describe('IPAC_rawdata_0610.csv')

y_num=2
featureNum=5
past=10
WindowsSize=30     #past +predict horizon= windowsize

training_set=pd.read_csv('IPAC_rawdata_0610.csv')
training_set_norm=data_norm(training_set)
training_set_norm["IPAC_+1"]=training_set_norm["IPAC"].shift(-1)
training_set_norm["WATER_+1"]=training_set_norm["WATER"].shift(-1)
training_set_norm=training_set_norm.dropna()
X_train, Y_train,X_val, Y_val=window_p(training_set_norm,"Train")
print("X_train: ",X_train.shape)
print("Y_train: ",Y_train.shape)

torch_dataset_train = PrepareData(X_train,Y_train)
train_loader = Data.DataLoader(
    dataset=torch_dataset_train,      # torch TensorDataset format
    batch_size=50,                    # mini batch size
    shuffle=True,                     # shuffle the data
    num_workers=0,                    # 多線程讀數據
    drop_last=True 
    )

torch_dataset_val = PrepareData(X_val, Y_val)
valid_loader = Data.DataLoader(
    dataset=torch_dataset_val,      
    batch_size=50,                  
    shuffle=True,                   
    num_workers=0,                  
    drop_last=True                  
    )

In [ ]:
#build the model and training 
model = softsensor(5,64,32,2,gpu=True)
model.to('cuda:0')
print(model)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',verbose=True,patience=3)

model, train_loss, valid_loss=train_model(model,  batch_size=50, patience=5, n_epochs=100,gpu=True)

In [ ]:
# visualize the loss as the network trained
fig = plt.figure(figsize=(8,6))
plt.plot(range(1,len(train_loss)+1),train_loss, label='Training Loss')
plt.plot(range(1,len(valid_loss)+1),valid_loss,label='Validation Loss')

# find position of lowest validation loss
minposs = valid_loss.index(min(valid_loss))+1 
plt.axvline(minposs, linestyle='--', color='r',label='Early Stopping Checkpoint')

plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(0, 0.03) # consistent scale
plt.xlim(0, len(train_loss)+1) # consistent scale
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
fig.savefig('loss_plot_64_32.png', bbox_inches='tight')

In [ ]:
#Model validation
testing_set=pd.read_csv('IPAC_valdata_0610.csv')
testing_set_norm=data_norm(testing_set)
testing_set_norm["IPAC_+1"]=testing_set_norm["IPAC"].shift(-1)
testing_set_norm["WATER_+1"]=testing_set_norm["WATER"].shift(-1)
testing_set_norm=testing_set_norm.dropna()
X_test, Y_test=window_p(testing_set_norm,"Test")
torch_dataset_test = PrepareData(X_test,Y_test)
test_loader = Data.DataLoader(
    dataset=torch_dataset_test,      # torch TensorDataset format
    batch_size=1,      # mini batch size              
    num_workers=0 # 多线程来读数据
)
for i in range(X_test.shape[0]):
    if i%50==0:
        print('-------------------------- {} -------------------------------------'.format(i))
        y_pred=model_test(model,test_loader,i,mode="inference")

In [ ]:
#MPC test

#import model
#model = softsensor(5,64,32,2,gpu=False)
#model.load_state_dict(torch.load("lstmcell_64_32_win10_0610.pt"))
#print(model)

testing_set=pd.read_csv('IPAC_valdata_0610.csv')
testing_set_norm=data_norm(testing_set)
testing_set_norm["IPAC_+1"]=testing_set_norm["IPAC"].shift(-1)
testing_set_norm["WATER_+1"]=testing_set_norm["WATER"].shift(-1)
testing_set_norm=testing_set_norm.dropna()
X_test, Y_test=window_p(testing_set_norm,"Test")
torch_dataset_test = PrepareData(X_test,Y_test)
test_loader = Data.DataLoader(
    dataset=torch_dataset_test,      # torch TensorDataset format
    batch_size=1,      # mini batch size              
    num_workers=0 # 多线程来读数据
)
for i in range(X_test.shape[0]):
    if i%50==0:
        print('-------------------------- {} -------------------------------------'.format(i))
        y_pred=model_mpc_test(model,test_loader,i,mode="mpc_test")